In [2]:
!git clone https://github.com/killerhack/VideoTo3dPoseAndBvh
import os
os.chdir('VideoTo3dPoseAndBvh')

Cloning into 'VideoTo3dPoseAndBvh'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 642 (delta 0), reused 2 (delta 0), pack-reused 637
Receiving objects: 100% (642/642), 88.00 MiB | 39.18 MiB/s, done.
Resolving deltas: 100% (150/150), done.


In [6]:
!gdown --id 1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW
!mv ./duc_se.pth ./joints_detectors/Alphapose/models/sppe
!gdown --id 1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC
!mv ./yolov3-spp.weights ./joints_detectors/Alphapose/models/yolo
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
!mv ./pretrained_h36m_detectron_coco.bin ./checkpoint


Downloading...
From: https://drive.google.com/uc?id=1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW
To: /content/VideoTo3dPoseAndBvh/duc_se.pth
239MB [00:05, 47.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC
To: /content/VideoTo3dPoseAndBvh/yolov3-spp.weights
252MB [00:04, 56.7MB/s]
--2021-03-04 00:21:39--  https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67892577 (65M) [application/octet-stream]
Saving to: ‘pretrained_h36m_detectron_coco.bin’

pretrained_h36m_det 100%[===================>]  64.75M  14.1MB/s    in 5.0s    

2021-03-04 00:21:44 (13.1 MB/s) - ‘pretrained_h36m_detectron_coco.bin’ saved [67892577/67892577]



In [8]:
os.chdir('/content/VideoTo3dPoseAndBvh/')

In [11]:
!python ./videopose.py

the video is 25.051 f/s
Loading YOLO model..
Loading pose model from joints_detectors/Alphapose/models/sppe/duc_se.pth
Start pose estimation...
100% 18/18 [00:01<00:00, 17.91it/s]
===========================> Rendering remaining images in the queue...
===========================> If this step takes too long, you can enable the --vis_fast flag to use fast rendering (real-time).
kpts npz save in  /content/VideoTo3dPoseAndBvh/outputs/outputvideo/alpha_pose_kunkun_cut_one_second/kunkun_cut_one_second.npz
-------------- load data spends 14.43 seconds
Loading checkpoint checkpoint/pretrained_h36m_detectron_coco.bin
-------------- load 3D model spends 0.04 seconds
Rendering...
-------------- generate reconstruction 3D data spends 0.18 seconds
total spend 0.000032 second
/content/VideoTo3dPoseAndBvh/outputs/inputvideo/kunkun_cut_one_second.mp4 --- elapsed time: 14.653919467999913 s
